In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv("train.csv")
train.head(3)

In [ ]:
test = pd.read_csv("test.csv")
test.head(3)

In [ ]:
pd.pivot_table(
    train,
    index="Pclass",
    columns="Survived",
    aggfunc="mean",
)

In [ ]:
pd.pivot_table(
    train,
    index="Sex",
    columns="Survived",
    aggfunc="mean",
)

In [ ]:
pd.pivot_table(
    train,
    index="SibSp",
    columns="Survived",
    aggfunc="mean",
    fill_value=0,
)

In [ ]:
pd.pivot_table(
    train,
    index="Parch",
    columns="Survived",
    aggfunc="mean",
    fill_value=0,
)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
survived_0 = train.loc[train["Survived"] == 0]
survived_1 = train.loc[train["Survived"] == 1]


sns.distplot(survived_0["Age"], label="survived_0");
sns.distplot(survived_1["Age"], label="survived_1");

plt.legend();

In [ ]:
pclass_1_survived_0 = train.loc[(train["Pclass"] == 1) & (train["Survived"] == 0)]
pclass_1_survived_1 = train.loc[(train["Pclass"] == 1) & (train["Survived"] == 1)]

pclass_2_survived_0 = train.loc[(train["Pclass"] == 2) & (train["Survived"] == 0)]
pclass_2_survived_1 = train.loc[(train["Pclass"] == 2) & (train["Survived"] == 1)]

pclass_3_survived_0 = train.loc[(train["Pclass"] == 3) & (train["Survived"] == 0)]
pclass_3_survived_1 = train.loc[(train["Pclass"] == 3) & (train["Survived"] == 1)]


In [ ]:
sns.distplot(pclass_1_survived_0["Age"], label="pclass_1_survived_0");
sns.distplot(pclass_1_survived_1["Age"], label="pclass_1_survived_1");
plt.legend();

In [ ]:
sns.distplot(pclass_2_survived_0["Age"], label="pclass_2_survived_0");
sns.distplot(pclass_2_survived_1["Age"], label="pclass_2_survived_1");
plt.legend();

In [ ]:
sns.distplot(pclass_3_survived_0["Age"], label="pclass_3_survived_0");
sns.distplot(pclass_3_survived_1["Age"], label="pclass_3_survived_1");
plt.legend();

In [ ]:
sns.pointplot(data=train, x="Pclass", y="Survived", hue="Sex");

In [ ]:
embarked_S = train.loc[train["Embarked"] == "S"]
embarked_C = train.loc[train["Embarked"] == "C"]
embarked_Q = train.loc[train["Embarked"] == "Q"]

In [ ]:
sns.pointplot(data=embarked_S, x="Pclass", y="Survived", hue="Sex");
plt.title("Embarked = S");

In [ ]:
sns.pointplot(data=embarked_C, x="Pclass", y="Survived", hue="Sex");
plt.title("Embarked = C");

In [ ]:
sns.pointplot(data=embarked_Q, x="Pclass", y="Survived", hue="Sex");
plt.title("Embarked = Q");

In [ ]:
sns.barplot(data=embarked_S, x="Sex", y="Fare", hue="Survived");
plt.title("Embarked = S");

In [ ]:
sns.barplot(data=embarked_C, x="Sex", y="Fare", hue="Survived");
plt.title("Embarked = C");

In [ ]:
sns.barplot(data=embarked_Q, x="Sex", y="Fare", hue="Survived");
plt.title("Embarked = Q");

In [ ]:
train.head(3)

In [ ]:
test.head(3)

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train["Age"].fillna(train["Age"].median(), inplace=True)
train["Embarked"].fillna(train["Embarked"].mode()[0], inplace=True)

test["Age"].fillna(test["Age"].median(), inplace=True)
test["Fare"].fillna(test["Fare"].median(), inplace=True)

drop_columns = ["PassengerId", "Cabin", "Ticket"]
train = train.drop(drop_columns, axis=1)
test = test.drop(drop_columns, axis=1)

print(train.isnull().sum())
print("-" * 10)
print(test.isnull().sum())

In [ ]:
train["family_size"]

train

train["age_bin"] = pd.cut(x=train["Age"], bins=5)
test["age_bin"] = pd.cut(x=test["Age"], bins=5)

print(train.shape, test.shape)
train.head(3)